Why is julia fast?

What makes a language slow?

![](xiaoqi-images/highway-memphis.jpg)

![](xiaoqi-images/union-memphis.jpg)

So, in order to find out why is Julia fast, we need to be asking a question: what makes a language slow?
let me give you a real life example: which one can you drive faster on, high way or local roads? 
On local roads, there are pedestrians and bikers, and all kinds of businesses that you need to be going to. 
The traffic condition is a lot more complicated than highway, where you only have to worry about the car ahead of you. 

Same goes for language, for a dynamic language, the compiler or interpreter have to be able to handle all types of input, floating point, integer, even user defined type. 
There is no way to predict what kind of input you will give, until the compiler evaluates that line of code. 
So you lose all the wonderful benefit of code optimization from the compiler. 
If you like to change variable types on the fly, mostly likely, that's why your code runs slow. 

# Essentially, type stability is the key to making a language fast. 

Julia achieves this type stability with JIT compiler and multiple dispatch. 
A compiler is a translator that turns source code to machine code. It is statically typed, but does not take in account runtime information 
An interpreter runs your code line by line. You can change the type of your variable on the fly, but is very slow. 
# JIT compiler:
JIT is a hybrid approach of the two. 
It compiles your code at runtime, so that it provide compiler runtime information for better optimization, and compiles the code to run very fast. 

Now that we understand the compiler choice of Julia, let's view some awesome design features that helps you understand your code better. 
- Multiple Dispatch: dynamic type achieved through type inference for easy use, static compile for speed 
- Cached compilation: don't invent wheel twice
- Easy code introspect with macro of every compilation stage.



# Multiple Dispatch + Type Inference
Multiple dispatch means a function or method can be dynamically dispatched based on the run time (dynamic) type. 
The easist way to understand multiple dispatch is that it is very much like function overload. 
Function overload is two functions that has the same name, but have different argument, either by type, or by number of argument.
This allows code to run the most efficient way for a specific type of input. 
But this is a lot of work to implement the same method for every possible data type, and a lot of time, it is redundant. 
Julia takes over the tedious task, and uses type inference to determin the type and generate efficient compiled code. It takes the burden of writing efficient code off your shoulder, let the compiler help you. 

In [1]:
my_square(x) = x^2
@code_typed my_square(1)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
└──      return %1
) => Int64

In [60]:
@code_typed my_square(1.0)

CodeInfo(
│╻╷ literal_pow1 1 ─ %1 = (Base.mul_float)(x, x)::Float64
│    └──      return %1
) => Float64

Of course, if you know the type of your input, it is always best practice to specify it. 

In [2]:
my_int_square(x::Int64) = x^2
@code_typed my_int_square(1)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
└──      return %1
) => Int64

# Cache and reuse compiled code.  
This is an easy concept. 
The code is only compiled once and then cached, so that you don't pay the compilation time again at the next call. 

In [74]:
@time 2^30

  0.003535 seconds (4.23 k allocations: 249.857 KiB)


1.073741824e9

In [75]:
@time 2^30

  0.000033 seconds (5 allocations: 176 bytes)


1.073741824e9

# Compilation Stages of Julia Code
Source Code -> AST (Macro Expansion) 
-> IR  -> (SSA) IR 
LLVM IR -> Native code 

In [24]:
@macroexpand 1+2

:(1 + 2)

In [15]:
@code_lowered 1+2

CodeInfo(
│53 1 ─ %1 = (Base.add_int)(x, y)
│   └──      return %1
)

In [16]:
@code_typed 1+2

CodeInfo(
│53 1 ─ %1 = (Base.add_int)(x, y)::Int64
│   └──      return %1
) => Int64

In [17]:
@code_llvm 1+2


; Function +
; Location: int.jl:53
define i64 @"julia_+_35442"(i64, i64) {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


In [21]:
# If you really want to see assembly code, do this:
@code_native 1+2

	.section	__TEXT,__text,regular,pure_instructions
; Function + {
; Location: int.jl:53
	decl	%eax
	leal	(%edi,%esi), %eax
	retl
;}
; Function <invalid> {
; Location: int.jl:53
	nopw	%cs:(%eax,%eax)
;}
